In [12]:
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
import os

from dotenv import load_dotenv
load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [13]:

from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

In [14]:
from langchain_core.documents import Document

text = """  ["Women's Father sAdamse когда There's Is Fliest Night in the Manlife Room gasps Imperial RO more of a story of peculiarity andgasp whippedidos up by the You won't get this wrong any other guy I just want to tell you how I feel You got to make you understand Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you We've known each other for so long Your heart's been aching but You're too shy to say it It's how we've all know what's been going on We know the game I'm with Gonna play it And if you ask me why I feel it You don't tell me you're too bad to see Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you Never gonna give you up Never gonna give you up Never gonna give you up We've known each other for so long Your heart's been aching but You're too shy to say It's how we've all know what's been going on We know the game I'm with Gonna play it I just wanna tell you how I feel You got to make you understand Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you Never gonna give you up Never gonna let you down Never gonna run around it He's hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you
"""

In [15]:

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Women', type='Person', properties={})]
Relationships:[]


In [16]:
from langchain_neo4j import Neo4jGraph
os.environ["NEO4J_URI"] = "neo4j+s://ffa05957.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "KYDXnv3miiiEFXu9p1119eo77ugFg_gcBqFxLlVD1h4"

graph = Neo4jGraph(refresh_schema=True)


In [17]:
graph.add_graph_documents(graph_documents, include_source=True)


In [18]:
from neo4j import GraphDatabase

# Neo4j credentials
NEO4J_URI = "neo4j+s://ffa05957.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "KYDXnv3miiiEFXu9p1119eo77ugFg_gcBqFxLlVD1h4"
# Create a class to interact with Neo4j
class Neo4jQueryTool:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def run_query(self, query):
        with self.driver.session() as session:
            result = session.run(query)
            return [record.data() for record in result]

# Create an instance of the query tool
tool = Neo4jQueryTool(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

# Test the tool with a sample query
try:
    print("Testing query tool...")
    
    # Replace with your test query
    sample_query = """
        MATCH (n)-[r]->(m)
        RETURN n, r, m
        LIMIT 10
    """
    
    results = tool.run_query(sample_query)
    for record in results:
        print(record)
finally:
    tool.close()


Testing query tool...
{'n': {'id': 'e1b070507716879d5c75a6117d566034', 'text': '  ["Women\'s Father sAdamse когда There\'s Is Fliest Night in the Manlife Room gasps Imperial RO more of a story of peculiarity andgasp whippedidos up by the You won\'t get this wrong any other guy I just want to tell you how I feel You got to make you understand Never gonna give you up Never gonna let you down Never gonna run around it He\'s hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you We\'ve known each other for so long Your heart\'s been aching but You\'re too shy to say it It\'s how we\'ve all know what\'s been going on We know the game I\'m with Gonna play it And if you ask me why I feel it You don\'t tell me you\'re too bad to see Never gonna give you up Never gonna let you down Never gonna run around it He\'s hurt you Never gonna make you cry Never gonna say goodbye Never gonna tell the lie And hurt you Never gonna give you up Never gonna let you dow

In [19]:
from langchain_neo4j import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Assuming 'graph' is your Neo4jGraph instance
qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True  # Explicitly allow dangerous requests
)

# Now you can run your query
response = qa_chain.run("Deep Neural Networks")
print(response)


[WARNING] C:\Users\SURYA\AppData\Local\Temp\ipykernel_26252\988719966.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("Deep Neural Networks")





> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)
WHERE n.name = "Deep Neural Networks"
RETURN n

Full Context:
[]

> Finished chain.
I am sorry, I don't know the answer.


In [23]:
# rag_faiss_system.py

# --- Dependencies ---
# Make sure you have installed the necessary packages:
# pip install langchain faiss-cpu sentence-transformers langchain-google-genai python-dotenv
# (or faiss-gpu if you have CUDA installed and want GPU acceleration)

import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS # Use langchain_community
from langchain_community.embeddings import HuggingFaceEmbeddings # Use langchain_community
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI # Import Gemini
from langchain.docstore.document import Document # For structured documents

# --- Configuration ---
# Load environment variables (especially GOOGLE_API_KEY)
load_dotenv() 

# Check if the API key is available
if os.getenv("GOOGLE_API_KEY") is None:
    print("Error: GOOGLE_API_KEY environment variable not set.")
    print("Please get an API key from Google AI Studio (https://aistudio.google.com/app/apikey)")
    print("and set it as an environment variable (e.g., in a .env file).")
    exit() # Exit if the key is missing

# --- Main Function ---
# Step 1: Initialize the embedding model
print("Initializing embedding model...")
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'} # Specify device (cpu or cuda)
    # Set encode_kwargs={'normalize_embeddings': True} if using cosine similarity,
    # but FAISS IndexFlatL2 uses L2 distance (Euclidean), so normalization is optional 
    # but often helpful. Let's keep it simple for L2.
)
print("Embedding model initialized.")

# Step 2 & 3: Prepare documents and create FAISS index
print("Preparing documents and creating FAISS vector store...")
documents_data = [
    {"content": """
        Neuro Prune: An Adaptive Approach for Efficient
Deep Neural Network Optimization on Edge
Devices
Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
1Professor, Department of Computer Science and Engineering, PSG College of Technology
grk.cse@psgtech.ac.in
2,3,4,5BE-CSE (AI&ML), PSG College of Technology
adhishthesak@gmail.com, mohanakumarp2828@gmail.com
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
Abstract—The optimization of deep neural networks for de-
ployment on edge devices is a significant research area due to
the demand for applications such as augmented reality, smart
cameras, and autonomous navigation. However, deploying large
deep learning models on edge devices poses challenges related to
computational power, energy consumption, and latency. Pruning
is a method to reduce the model size, accelerate inference, and
save power. The objective of the paper is to propose the Neuro
Prune algorithm and to apply it for the optimization of deep
neural networks on edge devices. Efforts have been made to
compare pruned and unpruned models. As a result, the pruned
model has an accuracy increase of 0.22%.
I. INTRODUCTION
The rise of intelligent systems, ranging from autonomous
drones to augmented reality (AR) devices and smart surveil-
lance cameras, has intensified the need for efficient deep neural
network deployment on edge devices with constrained com-
putational resources. While advanced deep learning models
like YOLO, SSD, ResNet, and MobileNet excel in accuracy,
their deployment on such devices is hindered by significant
computational demands, large memory footprints, and high
energy consumption. These challenges pose critical barriers to
real-time applications and sustainable deployment.
This paper builds on existing pruning techniques, includ-
ing unstructured pruning, structured pruning, and approaches
inspired by quantization, highlighting their strengths and lim-
itations. In response to the challenges identified, we introduce
Neuro Prune, a novel pruning approach that integrates princi-
ples from reinforcement learning and the Lottery Ticket Hy-
pothesis to achieve an optimal balance between accuracy, com-
putational efficiency, and energy consumption. By integrating
these principles, Neuro Prune provides a robust framework
for optimizing deep neural networks for deployment on edge
devices without sacrificing performance.
Neuro Prune reduces model size and computational re-
quirements while maintaining or even improving the model’s
accuracy. These findings underscore the potential of Neuro
Prune to enable real-time, energy-efficient deep learning on
resource-constrained devices.
Neuro Prune employs a systematic process to optimize
neural networks for edge deployment:
• Mask Initialization: Each layer’s weights are paired with
a binary mask (original_mask), initialized to ones.
• Activation Tracking: During forward passes over the
dataset, the magnitudes of weight activations are mon-
itored and recorded.
• Normalize and Prune: The recorded activations are
normalized, and a threshold based on the pruning fraction
is computed. Using this threshold, the masks are dynam-
ically updated to prune unimportant weights.
• Evaluate and Log Metrics: After pruning, metrics such
as model sparsity and accuracy are computed. A reward
metric is derived to balance accuracy and sparsity using
a tunable parameter, λweight.
• Retrain the Model: The pruned model undergoes retrain-
ing to improve the loss in accuracy while maintaining low
computational requirements
Neuro Prune’s unique combination of reinforcement learn-
ing for adaptive decision-making and the Lottery Ticket Hy-
pothesis for identifying optimal subnetworks enables it to
achieve remarkable efficiency. This approach ensures substan-
tial reduction in model size and energy consumption while
preserving accuracy and inference speed, making it an ideal
solution for edge-based deep learning applications.
Through extensive empirical evaluations and theoretical
insights, this paper demonstrates how Neuro Pruning outper-
forms traditional pruning methods, setting a new benchmark
for sustainable, high-performance AI deployment on edge
devices. By situating our work within the broader research
context, we aim to offer a critical contribution to the ongoing
evolution of neural network optimization techniques.
II. BACKGROUND
A. Need for Pruning Techniques
Pruning is a crucial optimization technique for deploying
deep neural networks in resource-constrained environments
like healthcare. For example, wearable devices like smart-
watches that monitor heart rates or portable EEG systems
for brain activity analysis require lightweight models due to
their limited computational power and memory. Pruning helps
address these constraints by reducing the memory footprint
and computational load, enabling efficient model deployment.
Additionally, it minimizes energy consumption, which is vital
for sustainable and real-time healthcare applications. Faster
inference enabled by pruning is particularly beneficial for
time-sensitive tasks like seizure detection or emergency di-
agnostics. Moreover, pruning enhances model interpretability
by retaining only the most critical components, which is
essential for gaining trust and ensuring regulatory compliance
in healthcare. By reducing the hardware and operational costs,
pruning also makes deploying advanced AI solutions on edge
devices more cost-effective and accessible.
B. Types of Pruning Techniques
Pruning methods are broadly classified based on granu-
larity, timing, and approach. Granularity-based pruning in-
cludes structured pruning, which removes entire components
such as neurons or filters to streamline computations, and
unstructured pruning, which eliminates individual weights
for finer optimization, though it often requires specialized
hardware. Timing-based pruning involves pre-training pruning,
which optimizes models before training; post-training prun-
ing, where redundant components are removed from trained
models with possible retraining; and dynamic pruning during
training, which adjusts models in real-time for improved
adaptability. Metric-based pruning uses parameters like weight
magnitude or gradient contribution to identify and remove
less critical components, while application-specific pruning
tailors strategies to specific tasks, such as optimizing for EEG
signal analysis, or hardware constraints like GPUs. Together,
these methods enable the development of efficient, scalable,
and task-specific models, particularly suited for edge device
deployment in healthcare and other domains.
III. RELATED WORK
Jielei Wang et al.[12] introduced an absorption pruning
method for object detection in remote sensing imagery. This
study achieves efficient compression with minimal accuracy
loss but requires careful layer-wise pruning ratio tuning.
Jan Muller et al., in their research[12], propose a neural
network pruning method for multi-object tracking (MOT) that
reduces model size by up to 70
Liang Li et al., in their work[9], introduce a novel pruning
method for DNNs that utilizes a self-adaptive mechanism
based on weight sparsity ratios and a protective reconstruction
mechanism. Their approach improves both model compres-
sion and accuracy, outperforming state-of-the-art methods on
CIFAR-10 and ImageNet datasets.
Giacomo Di Fabrizio et al.[4] discuss integrating machine
learning with edge devices like Google Coral AI and Nvidia
Jetson Nano. Their work revolutionizes computer vision and
real-time tracking but raises energy concerns due to the
computational demands of advanced algorithms.
Satoru Koda et al.[7] explore how weight pruning in
DNNs enhances out-of-distribution (OOD) detection, particu-
larly with Mahalanobis-based approaches, by improving global
feature extraction and leveraging weights not critical for
classification.
Shvetha S Kumar et al. analyze and compare three pruning
techniques—L1-norm filter pruning, channel pruning, and
weight pruning—on CNNs for accuracy and inference time,
highlighting greater speedup on NVIDIA V100 GPUs com-
pared to GTX1080 Ti in their study[8].
A Arun Prakash et al. propose a sub-path pruning algorithm
for solving the Minimum Robust-Cost Path (MRCP) problem
on stochastic networks, extending it to related problems and
demonstrating its efficiency on real-world networks in their
study[13].
Kyoungtaek Choi et al.[2] show that Quantization-Aware
Training outperforms Post-Training Quantization, and sparsity
training with Optimal Reduction pruning improves YOLOv4’s
efficiency and edge-device performance, despite potential
high-pruning rate drawbacks.
Viviana Crescitelli et al.[3] propose a filter-level pruning
method for DNNs. This approach optimizes inference speed
and resource efficiency on edge devices with minimal accuracy
loss but requires careful tuning of layer-specific pruning ratios.
Yongqi An et al.[1] introduce a retraining-free structured
pruning framework for Large Language Models (LLMs). Their
method incorporates a fluctuation-based pruning metric, adap-
tive structure search, and a bias compensation mechanism to
achieve efficient pruning. FLAP significantly enhances infer-
ence speed and reduces model size without requiring retrain-
ing, outperforming state-of-the-art methods like LLM-Pruner
and Wanda-sp on LLaMA models across various benchmarks.
From referring to these research papers, we derived an
idea for pruning on edge devices. Additionally, our proposed
solution is inspired by the research of Jaron Maene et al.[10],
which suggests that sparse subnetworks in dense networks can
achieve similar accuracy when retrained. This study demon-
strates stable training with linear mode connectivity, support-
ing the idea that lottery tickets retrain to similar regions, but
questions their independence from dense training and iterative
pruning.
Eran Malach et al.[11] further strengthen the lottery ticket
hypothesis by proving that over-parameterized neural networks
with random weights always contain a subnetwork matching
the accuracy of a target network, without additional training.
IV. NEUROLOGICAL PRUNING APPROACH
The Neurological Pruning Algorithm is designed to opti-
mize neural networks by iteratively pruning less important
weights while maintaining high performance. Unlike tradi-
tional methods, this approach incorporates adaptive, layer-
specific pruning thresholds that dynamically adjust based
on weight activations, ensuring efficient model optimization
across various stages of training. Drawing inspiration from
the Lottery Ticket Hypothesis, which suggests that sparse
subnetworks can perform similarly to their dense counterparts
when retrained, the algorithm selectively deactivates weights
using binary masks to enhance computational efficiency with-
out sacrificing accuracy.
The algorithm operates in several epochs, during which
weight activations are evaluated. Based on a quantile-based
strategy, pruning thresholds are established for each layer, en-
suring that only weights contributing minimally to the model’s
output are removed. After pruning, the model undergoes fine-
tuning to restore any lost accuracy and further optimize the
network. This step enhances the adaptability and scalability
of the pruning method, making it suitable for deployment in
resource-constrained environments
A. Key Characteristics and Advantages
The Neurological Pruning Algorithm boasts several key
characteristics that distinguish it from conventional pruning
techniques. One notable feature is its layer-specific pruning,
which evaluates the importance of weights individually for
each layer, leading to a more nuanced and efficient sparsifi-
cation process. This approach prevents the model from losing
essential features that may otherwise be pruned by a global
approach.
The algorithm’s use of quantile-based thresholding en-
sures that only weights with the least contribution to the
model’s output are pruned. This fine-tuned method avoids
performance degradation by carefully targeting weights that
are less significant to the network’s overall functionality.
Furthermore, the pruning process is iterative, allowing for
continuous model evaluation and real-time adjustments based
on performance feedback, ensuring that accuracy is retained
even as sparsity increases.
Additionally, the algorithm maintains a careful balance
between sparsity and accuracy, enabling a reduction in the
model’s computational complexity while preserving its predic-
tive capabilities. This characteristic is particularly important
for applications requiring rapid inference times, such as in
real-time healthcare systems and edge computing scenarios,
where both energy efficiency and computational power are
crucial.
B. Impact on Model Performance
The Neurological Pruning Algorithm significantly improves
the efficiency of neural networks, making them more suitable
for deployment on edge devices. Key performance metrics
such as compression ratio (CR) and accuracy retention
(AR) are critical indicators of the algorithm’s effectiveness.
For instance, in evaluating a ResNet-18 model, the algorithm
achieved a CR of 1, indicating that while the number of
parameters remained the same, the model’s efficiency im-
proved through selective pruning, reducing computational load
without sacrificing performance.
Moreover, the algorithm demonstrates the ability to achieve
high accuracy retention, with models maintaining or even
slightly improving their predictive performance post-pruning.
This outcome highlights the algorithm’s capability to effi-
ciently remove redundant weights while preserving the core
functionality of the network. The result is a model that is
not only more computationally efficient but also adaptable
across different architectures and datasets, making it suit-
able for a wide range of applications, from image classification
to natural language processing .
In practical terms, the Neurological Pruning Algorithm en-
ables faster inference times and lower energy consumption,
two crucial aspects for deploying AI models in resource-
constrained environments. As such, it holds significant poten-
tial for improving the viability of real-time, energy-efficient
AI applications, particularly in fields like healthcare, where
both speed and sustainability are essential.
V. NEUROLOGICAL PRUNING ALGORITHM
The Neurological Pruning Algorithm is a structured and
adaptive methodology designed to optimize neural networks
for resource-constrained environments. This algorithm goes
beyond traditional pruning approaches by incorporating layer-
specific pruning thresholds that dynamically adjust to the
importance of weight activations within each layer. Drawing
on principles inspired by the Lottery Ticket Hypothesis and re-
inforcement learning, the algorithm strikes an optimal balance
between model sparsity and accuracy .
A. Algorithm Description
The Neurological Pruning Algorithm iteratively refines the
neural network through a series of training epochs. During
each epoch, weight activations are tracked, normalized, and
analyzed to identify low-importance weights. These weights
are selectively deactivated using binary masks, ensuring com-
putational efficiency without compromising the network’s pre-
dictive performance. The algorithm further integrates a fine-
tuning phase, enabling the pruned network to recover and
enhance its accuracy. The detailed process is presented in
Algorithm 1.
By dynamically adjusting to layer-specific characteristics
and maintaining performance metrics, the Neurological Prun-
ing Algorithm ensures a significant reduction in computational
and memory requirements, paving the way for efficient deploy-
ment of deep learning models on edge devices .
B. Input Parameters and Output
Inputs:
• Pretrained model (M)
• Dataset (D)
• Prune fraction (p)
• Total epochs (E)
Output:
• Pruned and fine-tuned model (Mpruned)
C. Key Features and Advantages
The Neurological Pruning Algorithm is designed with sev-
eral distinctive features that contribute to its effectiveness
in optimizing neural network performance. These features
include:
Algorithm 1 Simplified Neurological Pruning Algorithm
Require: Pretrained model M, Dataset D, Prune fraction p,
Total epochs E
Ensure: Pruned and fine-tuned model Mpruned
1: Initialize binary masks M with all ones
2: for each epoch e = 1 to E do
3:
Compute weight activations A during forward passes
4:
for each layer i in M do
5:
Compute pruning threshold Ti as p-quantile of Ai
6:
Update mask Mi: retain weights with Ai > Ti
7:
Apply mask to layer: θi ←θi · Mi
8:
end for
9:
Evaluate model performance and update metrics
10: end for
11: Fine-tune Mpruned on D
12: return Mpruned
• Layer-specific Pruning: The algorithm applies pruning
on a per-layer basis, evaluating the weight activations
within each layer individually. This approach ensures that
the sparsity introduced by pruning is well-distributed and
context-dependent, facilitating more nuanced control over
model optimization .
• Quantile-based Thresholding: A quantile-based strat-
egy is employed to determine the pruning threshold for
each layer. This method ensures that only weights with
minimal contribution to the network’s output are pruned,
thereby reducing the risk of adversely affecting model
accuracy while enhancing sparsity .
• Iterative Pruning and Evaluation: The pruning pro-
cess follows an iterative approach, enabling continuous
assessment of the model’s performance after each pruning
step. This ensures dynamic adjustments to maintain a bal-
ance between model sparsity and predictive performance
throughout training.
• Preservation of Model Accuracy: By adjusting pruning
thresholds in response to ongoing performance metrics,
the algorithm ensures that model accuracy is not com-
promised. The careful tuning of these thresholds allows
for significant sparsity while maintaining the robustness
of the network’s predictive capabilities .
D. Impact on Neural Network Performance
The Neurological Pruning Algorithm has been extensively
evaluated across various benchmark datasets and model archi-
tectures, demonstrating its capacity to achieve the following
outcomes:
• High Sparsity with Minimal Accuracy Loss: The algo-
rithm effectively prunes unnecessary weights, achieving
high sparsity levels without causing significant degrada-
tion in model accuracy. This outcome is critical for the
efficient deployment of models in resource-constrained
environments.
• Reduced Computational Complexity: By sparsifying
the weight matrices, the algorithm reduces the compu-
tational load during inference, which translates to faster
model execution and lower resource consumption. This
characteristic is especially beneficial in edge computing
and real-time applications .
• Broad Applicability: The algorithm is highly flexible
and can be adapted to various neural network architec-
tures and datasets, making it suitable for a wide range
of tasks, from image recognition to natural language
processing .
VI. EVALUATION METRICS
A. Performance Comparison
Fig. 1. Evaluation of Neurological Pruning on ResNet-18 Model.
TABLE I
COMPARISON OF METRICS FOR UNPRUNED AND PRUNED MODELS
Metric
Unpruned
Model
Pruned
Model
Accuracy
0.9562
0.9583
Train Loss
0.1414
0.1625
Inference
Time
(s)
0.0318
0.0314
No of Parame-
ters(million)
11.7
11.7
B. Evaluation Metrics Calculation
The key metrics used to evaluate the proposed pruning
technique are as follows:
Compression Ratio (CR): The Compression Ratio (CR) is
calculated as:
CR = Total Parameters After Pruning
Total Parameters Before Pruning
(1)
CR = 11.7M
11.7M = 1
Accuracy Retention (AR): The Accuracy Retention (AR) is
calculated as:
AR =
Accuracy After Pruning −Baseline Accuracy
Baseline Accuracy

×100
(2)
AR =
0.958284 −0.956206
0.956206

× 100 = 0.22%
This result indicates that the pruned model achieves an ac-
curacy improvement of 0.22% when compared to unpruned
model.
C. Illustration of Results
Fig. 2. Graphical Representation of Neurological Pruning Results on ResNet-
18.
D. Discussion
The evaluation metrics highlight the effectiveness of the
proposed Neurological Pruning method on the ResNet-18
model. The Compression Ratio (CR) remains 1, indicating
no reduction in the total number of parameters, which suggests
that this approach optimizes the neural network by selectively
pruning connections rather than reducing the overall parameter
count. The Accuracy Retention (AR), which is approximately
0.22%, demonstrates that the Neurological Pruning technique
successfully improved the accuracy compared to the baseline
model.
These results validate the potential of Neurological Pruning
as an innovative approach to optimizing deep learning models.
By maintaining accuracy while likely enhancing sparsity and
computational efficiency, this method presents a promising di-
rection for achieving high-performance models under resource
constraints.
VII. CONCLUSION
In summary, the Neurological Pruning Algorithm provides
a systematic and principled approach to optimizing neural
networks. By balancing the trade-off between sparsity and
accuracy, it enables significant reductions in computational
complexity without sacrificing the network’s predictive per-
formance. Future work will focus on extending the algorithm
to dynamic network architectures and incorporating advanced
regularization techniques to further enhance its robustness and
adaptability in diverse applications.
REFERENCES
[1] Yongqi An, Xu Zhao, Tao Yu, Ming Tang, and Jinqiao Wang.
Fluctuation-based Adaptive Structured Pruning for Large Language
Models.
[2] Kyoungtaek Choi, Seong Min Wi, Ho Gi Jung, and Jae Kyu Suhr.
Simplification of Deep Neural Network-Based Object Detector for Real-
Time Edge Computing.
[3] Viviana Crescitelli, Seiji Miura, Goichi Ono, and Naohiro Kohmu. Edge
Devices Object Detection by Filter Pruning.
[4] Giacomo Di Fabrizio, Lorenzo Calisti, Chiara Contoli, Nicholas Kania,
and Emanuele Lattanzi. A Study on the Energy-Efficiency of the Object
Tracking Algorithms in Edge Devices.
[5] Yong He, Lin Lu, Hanjie Yuan, Guangxian Ye, Tianhang Jiang, Liang
Chen, Haiao Tan, Gaofeng Liao, Yanchao Zeng, and Jiawei Zhang. A
Compression Method for Object Detection Network for Edge Devices.
[6] Jielei Wang, Zongyong Cui, Zhipeng Zang, Xiangjie Meng, and Zongjie
Cao. Absorption Pruning of Deep Neural Network for Object Detection
in Remote Sensing Imagery.
[7] Satoru Koda, Alon Zolfi, Edita Grolman, Asaf Shabtai, Ikuya Morikawa,
and Yuval Elovici. Pros and Cons of Weight Pruning for Out-of-
Distribution Detection: An Empirical Survey.
[8] Shvetha S Kumar, Reshma R Nayak, Jismi S Kannampuzha, Jeeho Ryoo,
Sahil Rai, and Lizy K John. Evaluation of Pruning Techniques.
[9] Liang Li and Pengfei Zhao. Protective Self-Adaptive Pruning to Better
Compress DNNs.
[10] Jaron Maene, Mingxiao Li, and Marie-Francine Moens. Towards Un-
derstanding Iterative Magnitude Pruning: Why Lottery Tickets Win.
[11] Eran Malach, Gilad Yehudai, Shai Shalev-Shwartz, and Ohad Shamir.
Proving the Lottery Ticket Hypothesis: Pruning is All You Need.
[12] Jan M¨uller and Adrian Pigors. Efficient Multi-Object Tracking on Edge
Devices via Reconstruction-Based Channel Pruning.
[13] A. Arun Prakash and Karthik K. Srinivasan. Pruning Algorithms to
Determine Reliable Paths on Networks with Random and Correlated
Link Travel Times.
[14] Jose Reena K, Charul Nigam, G. Kirubasri, S. Jayachitra, Anurag Aeron,
and D. Suganthi. Real-Time Object Detection on Edge Devices Using
Mobile Neural Networks.
[15] Xubing Ye, Yukang Gan, Yixiao Ge, Xiao-Ping Zhang, and Yansong
Tang. ATP-LLaVA: Adaptive Token Pruning for Large Vision Language
Models.
[16] Shihao Zhang, Hekai Yang, Chunhua Yang, Wenxia Yuan, Xinghui
Li, Xinghua Wang, Yinsong Zhang, Xiaobo Cai, Yubo Sheng, Xiujuan
Deng, Wei Huang, Lei Li, Junjie He, and Baijuan Wang. Edge Device
Detection of Tea Leaves with One Bud and Two Leaves Based on
ShuffleNetv2-YOLOv5-Lite-E.
Neuro Prune: An Adaptive Approach for Efficient
Deep Neural Network Optimization on Edge
Devices
Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
1Professor, Department of Computer Science and Engineering, PSG College of Technology
grk.cse@psgtech.ac.in
2,3,4,5BE-CSE (AI&ML), PSG College of Technology
adhishthesak@gmail.com, mohanakumarp2828@gmail.com
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
Abstract—The optimization of deep neural networks for de-
ployment on edge devices is a significant research area due to
the demand for applications such as augmented reality, smart
cameras, and autonomous navigation. However, deploying large
deep learning models on edge devices poses challenges related to
computational power, energy consumption, and latency. Pruning
is a method to reduce the model size, accelerate inference, and
save power. The objective of the paper is to propose the Neuro
Prune algorithm and to apply it for the optimization of deep
neural networks on edge devices. Efforts have been made to
compare pruned and unpruned models. As a result, the pruned
model has an accuracy increase of 0.22%.
I. INTRODUCTION
The rise of intelligent systems, ranging from autonomous
drones to augmented reality (AR) devices and smart surveil-
lance cameras, has intensified the need for efficient deep neural
network deployment on edge devices with constrained com-
putational resources. While advanced deep learning models
like YOLO, SSD, ResNet, and MobileNet excel in accuracy,
their deployment on such devices is hindered by significant
computational demands, large memory footprints, and high
energy consumption. These challenges pose critical barriers to
real-time applications and sustainable deployment.
This paper builds on existing pruning techniques, includ-
ing unstructured pruning, structured pruning, and approaches
inspired by quantization, highlighting their strengths and lim-
itations. In response to the challenges identified, we introduce
Neuro Prune, a novel pruning approach that integrates princi-
ples from reinforcement learning and the Lottery Ticket Hy-
pothesis to achieve an optimal balance between accuracy, com-
putational efficiency, and energy consumption. By integrating
these principles, Neuro Prune provides a robust framework
for optimizing deep neural networks for deployment on edge
devices without sacrificing performance.
Neuro Prune reduces model size and computational re-
quirements while maintaining or even improving the model’s
accuracy. These findings underscore the potential of Neuro
Prune to enable real-time, energy-efficient deep learning on
resource-constrained devices.
Neuro Prune employs a systematic process to optimize
neural networks for edge deployment:
• Mask Initialization: Each layer’s weights are paired with
a binary mask (original_mask), initialized to ones.
• Activation Tracking: During forward passes over the
dataset, the magnitudes of weight activations are mon-
itored and recorded.
• Normalize and Prune: The recorded activations are
normalized, and a threshold based on the pruning fraction
is computed. Using this threshold, the masks are dynam-
ically updated to prune unimportant weights.
• Evaluate and Log Metrics: After pruning, metrics such
as model sparsity and accuracy are computed. A reward
metric is derived to balance accuracy and sparsity using
a tunable parameter, λweight.
• Retrain the Model: The pruned model undergoes retrain-
ing to improve the loss in accuracy while maintaining low
computational requirements
Neuro Prune’s unique combination of reinforcement learn-
ing for adaptive decision-making and the Lottery Ticket Hy-
pothesis for identifying optimal subnetworks enables it to
achieve remarkable efficiency. This approach ensures substan-
tial reduction in model size and energy consumption while
preserving accuracy and inference speed, making it an ideal
solution for edge-based deep learning applications.
Through extensive empirical evaluations and theoretical
insights, this paper demonstrates how Neuro Pruning outper-
forms traditional pruning methods, setting a new benchmark
for sustainable, high-performance AI deployment on edge
devices. By situating our work within the broader research
context, we aim to offer a critical contribution to the ongoing
evolution of neural network optimization techniques.
II. BACKGROUND
A. Need for Pruning Techniques
Pruning is a crucial optimization technique for deploying
deep neural networks in resource-constrained environments
like healthcare. For example, wearable devices like smart-
watches that monitor heart rates or portable EEG systems
for brain activity analysis require lightweight models due to
their limited computational power and memory. Pruning helps
address these constraints by reducing the memory footprint
and computational load, enabling efficient model deployment.
Additionally, it minimizes energy consumption, which is vital
for sustainable and real-time healthcare applications. Faster
inference enabled by pruning is particularly beneficial for
time-sensitive tasks like seizure detection or emergency di-
agnostics. Moreover, pruning enhances model interpretability
by retaining only the most critical components, which is
essential for gaining trust and ensuring regulatory compliance
in healthcare. By reducing the hardware and operational costs,
pruning also makes deploying advanced AI solutions on edge
devices more cost-effective and accessible.
B. Types of Pruning Techniques
Pruning methods are broadly classified based on granu-
larity, timing, and approach. Granularity-based pruning in-
cludes structured pruning, which removes entire components
such as neurons or filters to streamline computations, and
unstructured pruning, which eliminates individual weights
for finer optimization, though it often requires specialized
hardware. Timing-based pruning involves pre-training pruning,
which optimizes models before training; post-training prun-
ing, where redundant components are removed from trained
models with possible retraining; and dynamic pruning during
training, which adjusts models in real-time for improved
adaptability. Metric-based pruning uses parameters like weight
magnitude or gradient contribution to identify and remove
less critical components, while application-specific pruning
tailors strategies to specific tasks, such as optimizing for EEG
signal analysis, or hardware constraints like GPUs. Together,
these methods enable the development of efficient, scalable,
and task-specific models, particularly suited for edge device
deployment in healthcare and other domains.
III. RELATED WORK
Jielei Wang et al.[12] introduced an absorption pruning
method for object detection in remote sensing imagery. This
study achieves efficient compression with minimal accuracy
loss but requires careful layer-wise pruning ratio tuning.
Jan Muller et al., in their research[12], propose a neural
network pruning method for multi-object tracking (MOT) that
reduces model size by up to 70
Liang Li et al., in their work[9], introduce a novel pruning
method for DNNs that utilizes a self-adaptive mechanism
based on weight sparsity ratios and a protective reconstruction
mechanism. Their approach improves both model compres-
sion and accuracy, outperforming state-of-the-art methods on
CIFAR-10 and ImageNet datasets.
Giacomo Di Fabrizio et al.[4] discuss integrating machine
learning with edge devices like Google Coral AI and Nvidia
Jetson Nano. Their work revolutionizes computer vision and
real-time tracking but raises energy concerns due to the
computational demands of advanced algorithms.
Satoru Koda et al.[7] explore how weight pruning in
DNNs enhances out-of-distribution (OOD) detection, particu-
larly with Mahalanobis-based approaches, by improving global
feature extraction and leveraging weights not critical for
classification.
Shvetha S Kumar et al. analyze and compare three pruning
techniques—L1-norm filter pruning, channel pruning, and
weight pruning—on CNNs for accuracy and inference time,
highlighting greater speedup on NVIDIA V100 GPUs com-
pared to GTX1080 Ti in their study[8].
A Arun Prakash et al. propose a sub-path pruning algorithm
for solving the Minimum Robust-Cost Path (MRCP) problem
on stochastic networks, extending it to related problems and
demonstrating its efficiency on real-world networks in their
study[13].
Kyoungtaek Choi et al.[2] show that Quantization-Aware
Training outperforms Post-Training Quantization, and sparsity
training with Optimal Reduction pruning improves YOLOv4’s
efficiency and edge-device performance, despite potential
high-pruning rate drawbacks.
Viviana Crescitelli et al.[3] propose a filter-level pruning
method for DNNs. This approach optimizes inference speed
and resource efficiency on edge devices with minimal accuracy
loss but requires careful tuning of layer-specific pruning ratios.
Yongqi An et al.[1] introduce a retraining-free structured
pruning framework for Large Language Models (LLMs). Their
method incorporates a fluctuation-based pruning metric, adap-
tive structure search, and a bias compensation mechanism to
achieve efficient pruning. FLAP significantly enhances infer-
ence speed and reduces model size without requiring retrain-
ing, outperforming state-of-the-art methods like LLM-Pruner
and Wanda-sp on LLaMA models across various benchmarks.
From referring to these research papers, we derived an
idea for pruning on edge devices. Additionally, our proposed
solution is inspired by the research of Jaron Maene et al.[10],
which suggests that sparse subnetworks in dense networks can
achieve similar accuracy when retrained. This study demon-
strates stable training with linear mode connectivity, support-
ing the idea that lottery tickets retrain to similar regions, but
questions their independence from dense training and iterative
pruning.
Eran Malach et al.[11] further strengthen the lottery ticket
hypothesis by proving that over-parameterized neural networks
with random weights always contain a subnetwork matching
the accuracy of a target network, without additional training.
IV. NEUROLOGICAL PRUNING APPROACH
The Neurological Pruning Algorithm is designed to opti-
mize neural networks by iteratively pruning less important
weights while maintaining high performance. Unlike tradi-
tional methods, this approach incorporates adaptive, layer-
specific pruning thresholds that dynamically adjust based
on weight activations, ensuring efficient model optimization
across various stages of training. Drawing inspiration from
the Lottery Ticket Hypothesis, which suggests that sparse
subnetworks can perform similarly to their dense counterparts
when retrained, the algorithm selectively deactivates weights
using binary masks to enhance computational efficiency with-
out sacrificing accuracy.
The algorithm operates in several epochs, during which
weight activations are evaluated. Based on a quantile-based
strategy, pruning thresholds are established for each layer, en-
suring that only weights contributing minimally to the model’s
output are removed. After pruning, the model undergoes fine-
tuning to restore any lost accuracy and further optimize the
network. This step enhances the adaptability and scalability
of the pruning method, making it suitable for deployment in
resource-constrained environments
A. Key Characteristics and Advantages
The Neurological Pruning Algorithm boasts several key
characteristics that distinguish it from conventional pruning
techniques. One notable feature is its layer-specific pruning,
which evaluates the importance of weights individually for
each layer, leading to a more nuanced and efficient sparsifi-
cation process. This approach prevents the model from losing
essential features that may otherwise be pruned by a global
approach.
The algorithm’s use of quantile-based thresholding en-
sures that only weights with the least contribution to the
model’s output are pruned. This fine-tuned method avoids
performance degradation by carefully targeting weights that
are less significant to the network’s overall functionality.
Furthermore, the pruning process is iterative, allowing for
continuous model evaluation and real-time adjustments based
on performance feedback, ensuring that accuracy is retained
even as sparsity increases.
Additionally, the algorithm maintains a careful balance
between sparsity and accuracy, enabling a reduction in the
model’s computational complexity while preserving its predic-
tive capabilities. This characteristic is particularly important
for applications requiring rapid inference times, such as in
real-time healthcare systems and edge computing scenarios,
where both energy efficiency and computational power are
crucial.
B. Impact on Model Performance
The Neurological Pruning Algorithm significantly improves
the efficiency of neural networks, making them more suitable
for deployment on edge devices. Key performance metrics
such as compression ratio (CR) and accuracy retention
(AR) are critical indicators of the algorithm’s effectiveness.
For instance, in evaluating a ResNet-18 model, the algorithm
achieved a CR of 1, indicating that while the number of
parameters remained the same, the model’s efficiency im-
proved through selective pruning, reducing computational load
without sacrificing performance.
Moreover, the algorithm demonstrates the ability to achieve
high accuracy retention, with models maintaining or even
slightly improving their predictive performance post-pruning.
This outcome highlights the algorithm’s capability to effi-
ciently remove redundant weights while preserving the core
functionality of the network. The result is a model that is
not only more computationally efficient but also adaptable
across different architectures and datasets, making it suit-
able for a wide range of applications, from image classification
to natural language processing .
In practical terms, the Neurological Pruning Algorithm en-
ables faster inference times and lower energy consumption,
two crucial aspects for deploying AI models in resource-
constrained environments. As such, it holds significant poten-
tial for improving the viability of real-time, energy-efficient
AI applications, particularly in fields like healthcare, where
both speed and sustainability are essential.
V. NEUROLOGICAL PRUNING ALGORITHM
The Neurological Pruning Algorithm is a structured and
adaptive methodology designed to optimize neural networks
for resource-constrained environments. This algorithm goes
beyond traditional pruning approaches by incorporating layer-
specific pruning thresholds that dynamically adjust to the
importance of weight activations within each layer. Drawing
on principles inspired by the Lottery Ticket Hypothesis and re-
inforcement learning, the algorithm strikes an optimal balance
between model sparsity and accuracy .
A. Algorithm Description
The Neurological Pruning Algorithm iteratively refines the
neural network through a series of training epochs. During
each epoch, weight activations are tracked, normalized, and
analyzed to identify low-importance weights. These weights
are selectively deactivated using binary masks, ensuring com-
putational efficiency without compromising the network’s pre-
dictive performance. The algorithm further integrates a fine-
tuning phase, enabling the pruned network to recover and
enhance its accuracy. The detailed process is presented in
Algorithm 1.
By dynamically adjusting to layer-specific characteristics
and maintaining performance metrics, the Neurological Prun-
ing Algorithm ensures a significant reduction in computational
and memory requirements, paving the way for efficient deploy-
ment of deep learning models on edge devices .
B. Input Parameters and Output
Inputs:
• Pretrained model (M)
• Dataset (D)
• Prune fraction (p)
• Total epochs (E)
Output:
• Pruned and fine-tuned model (Mpruned)
C. Key Features and Advantages
The Neurological Pruning Algorithm is designed with sev-
eral distinctive features that contribute to its effectiveness
in optimizing neural network performance. These features
include:
Algorithm 1 Simplified Neurological Pruning Algorithm
Require: Pretrained model M, Dataset D, Prune fraction p,
Total epochs E
Ensure: Pruned and fine-tuned model Mpruned
1: Initialize binary masks M with all ones
2: for each epoch e = 1 to E do
3:
Compute weight activations A during forward passes
4:
for each layer i in M do
5:
Compute pruning threshold Ti as p-quantile of Ai
6:
Update mask Mi: retain weights with Ai > Ti
7:
Apply mask to layer: θi ←θi · Mi
8:
end for
9:
Evaluate model performance and update metrics
10: end for
11: Fine-tune Mpruned on D
12: return Mpruned
• Layer-specific Pruning: The algorithm applies pruning
on a per-layer basis, evaluating the weight activations
within each layer individually. This approach ensures that
the sparsity introduced by pruning is well-distributed and
context-dependent, facilitating more nuanced control over
model optimization .
• Quantile-based Thresholding: A quantile-based strat-
egy is employed to determine the pruning threshold for
each layer. This method ensures that only weights with
minimal contribution to the network’s output are pruned,
thereby reducing the risk of adversely affecting model
accuracy while enhancing sparsity .
• Iterative Pruning and Evaluation: The pruning pro-
cess follows an iterative approach, enabling continuous
assessment of the model’s performance after each pruning
step. This ensures dynamic adjustments to maintain a bal-
ance between model sparsity and predictive performance
throughout training.
• Preservation of Model Accuracy: By adjusting pruning
thresholds in response to ongoing performance metrics,
the algorithm ensures that model accuracy is not com-
promised. The careful tuning of these thresholds allows
for significant sparsity while maintaining the robustness
of the network’s predictive capabilities .
D. Impact on Neural Network Performance
The Neurological Pruning Algorithm has been extensively
evaluated across various benchmark datasets and model archi-
tectures, demonstrating its capacity to achieve the following
outcomes:
• High Sparsity with Minimal Accuracy Loss: The algo-
rithm effectively prunes unnecessary weights, achieving
high sparsity levels without causing significant degrada-
tion in model accuracy. This outcome is critical for the
efficient deployment of models in resource-constrained
environments.
• Reduced Computational Complexity: By sparsifying
the weight matrices, the algorithm reduces the compu-
tational load during inference, which translates to faster
model execution and lower resource consumption. This
characteristic is especially beneficial in edge computing
and real-time applications .
• Broad Applicability: The algorithm is highly flexible
and can be adapted to various neural network architec-
tures and datasets, making it suitable for a wide range
of tasks, from image recognition to natural language
processing .
VI. EVALUATION METRICS
A. Performance Comparison
Fig. 1. Evaluation of Neurological Pruning on ResNet-18 Model.
TABLE I
COMPARISON OF METRICS FOR UNPRUNED AND PRUNED MODELS
Metric
Unpruned
Model
Pruned
Model
Accuracy
0.9562
0.9583
Train Loss
0.1414
0.1625
Inference
Time
(s)
0.0318
0.0314
No of Parame-
ters(million)
11.7
11.7
B. Evaluation Metrics Calculation
The key metrics used to evaluate the proposed pruning
technique are as follows:
Compression Ratio (CR): The Compression Ratio (CR) is
calculated as:
CR = Total Parameters After Pruning
Total Parameters Before Pruning
(1)
CR = 11.7M
11.7M = 1
Accuracy Retention (AR): The Accuracy Retention (AR) is
calculated as:
AR =
Accuracy After Pruning −Baseline Accuracy
Baseline Accuracy

×100
(2)
AR =
0.958284 −0.956206
0.956206

× 100 = 0.22%
This result indicates that the pruned model achieves an ac-
curacy improvement of 0.22% when compared to unpruned
model.
C. Illustration of Results
Fig. 2. Graphical Representation of Neurological Pruning Results on ResNet-
18.
D. Discussion
The evaluation metrics highlight the effectiveness of the
proposed Neurological Pruning method on the ResNet-18
model. The Compression Ratio (CR) remains 1, indicating
no reduction in the total number of parameters, which suggests
that this approach optimizes the neural network by selectively
pruning connections rather than reducing the overall parameter
count. The Accuracy Retention (AR), which is approximately
0.22%, demonstrates that the Neurological Pruning technique
successfully improved the accuracy compared to the baseline
model.
These results validate the potential of Neurological Pruning
as an innovative approach to optimizing deep learning models.
By maintaining accuracy while likely enhancing sparsity and
computational efficiency, this method presents a promising di-
rection for achieving high-performance models under resource
constraints.
VII. CONCLUSION
In summary, the Neurological Pruning Algorithm provides
a systematic and principled approach to optimizing neural
networks. By balancing the trade-off between sparsity and
accuracy, it enables significant reductions in computational
complexity without sacrificing the network’s predictive per-
formance. Future work will focus on extending the algorithm
to dynamic network architectures and incorporating advanced
regularization techniques to further enhance its robustness and
adaptability in diverse applications.
REFERENCES
[1] Yongqi An, Xu Zhao, Tao Yu, Ming Tang, and Jinqiao Wang.
Fluctuation-based Adaptive Structured Pruning for Large Language
Models.
[2] Kyoungtaek Choi, Seong Min Wi, Ho Gi Jung, and Jae Kyu Suhr.
Simplification of Deep Neural Network-Based Object Detector for Real-
Time Edge Computing.
[3] Viviana Crescitelli, Seiji Miura, Goichi Ono, and Naohiro Kohmu. Edge
Devices Object Detection by Filter Pruning.
[4] Giacomo Di Fabrizio, Lorenzo Calisti, Chiara Contoli, Nicholas Kania,
and Emanuele Lattanzi. A Study on the Energy-Efficiency of the Object
Tracking Algorithms in Edge Devices.
[5] Yong He, Lin Lu, Hanjie Yuan, Guangxian Ye, Tianhang Jiang, Liang
Chen, Haiao Tan, Gaofeng Liao, Yanchao Zeng, and Jiawei Zhang. A
Compression Method for Object Detection Network for Edge Devices.
[6] Jielei Wang, Zongyong Cui, Zhipeng Zang, Xiangjie Meng, and Zongjie
Cao. Absorption Pruning of Deep Neural Network for Object Detection
in Remote Sensing Imagery.
[7] Satoru Koda, Alon Zolfi, Edita Grolman, Asaf Shabtai, Ikuya Morikawa,
and Yuval Elovici. Pros and Cons of Weight Pruning for Out-of-
Distribution Detection: An Empirical Survey.
[8] Shvetha S Kumar, Reshma R Nayak, Jismi S Kannampuzha, Jeeho Ryoo,
Sahil Rai, and Lizy K John. Evaluation of Pruning Techniques.
[9] Liang Li and Pengfei Zhao. Protective Self-Adaptive Pruning to Better
Compress DNNs.
[10] Jaron Maene, Mingxiao Li, and Marie-Francine Moens. Towards Un-
derstanding Iterative Magnitude Pruning: Why Lottery Tickets Win.
[11] Eran Malach, Gilad Yehudai, Shai Shalev-Shwartz, and Ohad Shamir.
Proving the Lottery Ticket Hypothesis: Pruning is All You Need.
[12] Jan M¨uller and Adrian Pigors. Efficient Multi-Object Tracking on Edge
Devices via Reconstruction-Based Channel Pruning.
[13] A. Arun Prakash and Karthik K. Srinivasan. Pruning Algorithms to
Determine Reliable Paths on Networks with Random and Correlated
Link Travel Times.
[14] Jose Reena K, Charul Nigam, G. Kirubasri, S. Jayachitra, Anurag Aeron,
and D. Suganthi. Real-Time Object Detection on Edge Devices Using
Mobile Neural Networks.
[15] Xubing Ye, Yukang Gan, Yixiao Ge, Xiao-Ping Zhang, and Yansong
Tang. ATP-LLaVA: Adaptive Token Pruning for Large Vision Language
Models.
[16] Shihao Zhang, Hekai Yang, Chunhua Yang, Wenxia Yuan, Xinghui
Li, Xinghua Wang, Yinsong Zhang, Xiaobo Cai, Yubo Sheng, Xiujuan
Deng, Wei Huang, Lei Li, Junjie He, and Baijuan Wang. Edge Device
Detection of Tea Leaves with One Bud and Two Leaves Based on
ShuffleNetv2-YOLOv5-Lite-E.
""", "metadata": {"source": "geo_facts.txt", "page": 1}},
    {"content": "The Great Wall of China is located in China and is a famous landmark.", "metadata": {"source": "world_landmarks.txt", "page": 5}},
    {"content": "Paris is known for the Eiffel Tower.", "metadata": {"source": "geo_facts.txt", "page": 2}},
]

# Convert raw data to LangChain Document objects
documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in documents_data]

# Create FAISS vector store directly from documents
# This handles embedding the texts and building the index internally
try:
    vector_store = FAISS.from_documents(documents, embedding_model)
    print("FAISS vector store created successfully.")
except Exception as e:
    print(f"Error creating FAISS vector store: {e}")
# Exit if vector store creation fails

# Step 4: Load Gemini model for text generation

# Step 5: Create the Retrieval-Augmented Generation pipeline
print("Creating RAG chain...")
retriever = vector_store.as_retriever(
    search_type="similarity", # Other options: "mmr", "similarity_score_threshold"
    search_kwargs={'k': 2} # Retrieve top 2 relevant documents
)

# Use the recommended from_chain_type method
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # Options: "stuff", "map_reduce", "refine", "map_rerank"
                        # "stuff" puts all retrieved docs into the context
    retriever=retriever,
    return_source_documents=True # Include source documents in the output
    # chain_type_kwargs={"prompt": YOUR_CUSTOM_PROMPT} # Optional: customize prompt
)
print("RAG chain created.")


[INFO] Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Initializing embedding model...
Embedding model initialized.
Preparing documents and creating FAISS vector store...
FAISS vector store created successfully.
Creating RAG chain...
RAG chain created.


In [ ]:
# rag_faiss_system.py

# --- Dependencies ---
# Make sure you have installed the necessary packages:
# pip install langchain faiss-cpu sentence-transformers langchain-google-genai python-dotenv
# (or faiss-gpu if you have CUDA installed and want GPU acceleration)

import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS # Use langchain_community
from langchain_community.embeddings import HuggingFaceEmbeddings # Use langchain_community
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI # Import Gemini
from langchain.docstore.document import Document # For structured documents

# --- Configuration ---
# Load environment variables (especially GOOGLE_API_KEY)
load_dotenv() 

# Check if the API key is available
if os.getenv("GOOGLE_API_KEY") is None:
    print("Error: GOOGLE_API_KEY environment variable not set.")
    print("Please get an API key from Google AI Studio (https://aistudio.google.com/app/apikey)")
    print("and set it as an environment variable (e.g., in a .env file).")
    exit() # Exit if the key is missing

# --- Main Function ---
# Step 1: Initialize the embedding model
print("Initializing embedding model...")
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'} # Specify device (cpu or cuda)
    # Set encode_kwargs={'normalize_embeddings': True} if using cosine similarity,
    # but FAISS IndexFlatL2 uses L2 distance (Euclidean), so normalization is optional 
    # but often helpful. Let's keep it simple for L2.
)
print("Embedding model initialized.")

# Step 2 & 3: Prepare documents and create FAISS index
print("Preparing documents and creating FAISS vector store...")
documents_data = [
    {"content": """
        Neuro Prune: An Adaptive Approach for Efficient
Deep Neural Network Optimization on Edge
Devices
Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
1Professor, Department of Computer Science and Engineering, PSG College of Technology
grk.cse@psgtech.ac.in
2,3,4,5BE-CSE (AI&ML), PSG College of Technology
adhishthesak@gmail.com, mohanakumarp2828@gmail.com
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
Abstract—The optimization of deep neural networks for de-
ployment on edge devices is a significant research area due to
the demand for applications such as augmented reality, smart
cameras, and autonomous navigation. However, deploying large
deep learning models on edge devices poses challenges related to
computational power, energy consumption, and latency. Pruning
is a method to reduce the model size, accelerate inference, and
save power. The objective of the paper is to propose the Neuro
Prune algorithm and to apply it for the optimization of deep
neural networks on edge devices. Efforts have been made to
compare pruned and unpruned models. As a result, the pruned
model has an accuracy increase of 0.22%.
I. INTRODUCTION
The rise of intelligent systems, ranging from autonomous
drones to augmented reality (AR) devices and smart surveil-
lance cameras, has intensified the need for efficient deep neural
network deployment on edge devices with constrained com-
putational resources. While advanced deep learning models
like YOLO, SSD, ResNet, and MobileNet excel in accuracy,
their deployment on such devices is hindered by significant
computational demands, large memory footprints, and high
energy consumption. These challenges pose critical barriers to
real-time applications and sustainable deployment.
This paper builds on existing pruning techniques, includ-
ing unstructured pruning, structured pruning, and approaches
inspired by quantization, highlighting their strengths and lim-
itations. In response to the challenges identified, we introduce
Neuro Prune, a novel pruning approach that integrates princi-
ples from reinforcement learning and the Lottery Ticket Hy-
pothesis to achieve an optimal balance between accuracy, com-
putational efficiency, and energy consumption. By integrating
these principles, Neuro Prune provides a robust framework
for optimizing deep neural networks for deployment on edge
devices without sacrificing performance.
Neuro Prune reduces model size and computational re-
quirements while maintaining or even improving the model’s
accuracy. These findings underscore the potential of Neuro
Prune to enable real-time, energy-efficient deep learning on
resource-constrained devices.
Neuro Prune employs a systematic process to optimize
neural networks for edge deployment:
• Mask Initialization: Each layer’s weights are paired with
a binary mask (original_mask), initialized to ones.
• Activation Tracking: During forward passes over the
dataset, the magnitudes of weight activations are mon-
itored and recorded.
• Normalize and Prune: The recorded activations are
normalized, and a threshold based on the pruning fraction
is computed. Using this threshold, the masks are dynam-
ically updated to prune unimportant weights.
• Evaluate and Log Metrics: After pruning, metrics such
as model sparsity and accuracy are computed. A reward
metric is derived to balance accuracy and sparsity using
a tunable parameter, λweight.
• Retrain the Model: The pruned model undergoes retrain-
ing to improve the loss in accuracy while maintaining low
computational requirements
Neuro Prune’s unique combination of reinforcement learn-
ing for adaptive decision-making and the Lottery Ticket Hy-
pothesis for identifying optimal subnetworks enables it to
achieve remarkable efficiency. This approach ensures substan-
tial reduction in model size and energy consumption while
preserving accuracy and inference speed, making it an ideal
solution for edge-based deep learning applications.
Through extensive empirical evaluations and theoretical
insights, this paper demonstrates how Neuro Pruning outper-
forms traditional pruning methods, setting a new benchmark
for sustainable, high-performance AI deployment on edge
devices. By situating our work within the broader research
context, we aim to offer a critical contribution to the ongoing
evolution of neural network optimization techniques.
II. BACKGROUND
A. Need for Pruning Techniques
Pruning is a crucial optimization technique for deploying
deep neural networks in resource-constrained environments
like healthcare. For example, wearable devices like smart-
watches that monitor heart rates or portable EEG systems
for brain activity analysis require lightweight models due to
their limited computational power and memory. Pruning helps
address these constraints by reducing the memory footprint
and computational load, enabling efficient model deployment.
Additionally, it minimizes energy consumption, which is vital
for sustainable and real-time healthcare applications. Faster
inference enabled by pruning is particularly beneficial for
time-sensitive tasks like seizure detection or emergency di-
agnostics. Moreover, pruning enhances model interpretability
by retaining only the most critical components, which is
essential for gaining trust and ensuring regulatory compliance
in healthcare. By reducing the hardware and operational costs,
pruning also makes deploying advanced AI solutions on edge
devices more cost-effective and accessible.
B. Types of Pruning Techniques
Pruning methods are broadly classified based on granu-
larity, timing, and approach. Granularity-based pruning in-
cludes structured pruning, which removes entire components
such as neurons or filters to streamline computations, and
unstructured pruning, which eliminates individual weights
for finer optimization, though it often requires specialized
hardware. Timing-based pruning involves pre-training pruning,
which optimizes models before training; post-training prun-
ing, where redundant components are removed from trained
models with possible retraining; and dynamic pruning during
training, which adjusts models in real-time for improved
adaptability. Metric-based pruning uses parameters like weight
magnitude or gradient contribution to identify and remove
less critical components, while application-specific pruning
tailors strategies to specific tasks, such as optimizing for EEG
signal analysis, or hardware constraints like GPUs. Together,
these methods enable the development of efficient, scalable,
and task-specific models, particularly suited for edge device
deployment in healthcare and other domains.
III. RELATED WORK
Jielei Wang et al.[12] introduced an absorption pruning
method for object detection in remote sensing imagery. This
study achieves efficient compression with minimal accuracy
loss but requires careful layer-wise pruning ratio tuning.
Jan Muller et al., in their research[12], propose a neural
network pruning method for multi-object tracking (MOT) that
reduces model size by up to 70
Liang Li et al., in their work[9], introduce a novel pruning
method for DNNs that utilizes a self-adaptive mechanism
based on weight sparsity ratios and a protective reconstruction
mechanism. Their approach improves both model compres-
sion and accuracy, outperforming state-of-the-art methods on
CIFAR-10 and ImageNet datasets.
Giacomo Di Fabrizio et al.[4] discuss integrating machine
learning with edge devices like Google Coral AI and Nvidia
Jetson Nano. Their work revolutionizes computer vision and
real-time tracking but raises energy concerns due to the
computational demands of advanced algorithms.
Satoru Koda et al.[7] explore how weight pruning in
DNNs enhances out-of-distribution (OOD) detection, particu-
larly with Mahalanobis-based approaches, by improving global
feature extraction and leveraging weights not critical for
classification.
Shvetha S Kumar et al. analyze and compare three pruning
techniques—L1-norm filter pruning, channel pruning, and
weight pruning—on CNNs for accuracy and inference time,
highlighting greater speedup on NVIDIA V100 GPUs com-
pared to GTX1080 Ti in their study[8].
A Arun Prakash et al. propose a sub-path pruning algorithm
for solving the Minimum Robust-Cost Path (MRCP) problem
on stochastic networks, extending it to related problems and
demonstrating its efficiency on real-world networks in their
study[13].
Kyoungtaek Choi et al.[2] show that Quantization-Aware
Training outperforms Post-Training Quantization, and sparsity
training with Optimal Reduction pruning improves YOLOv4’s
efficiency and edge-device performance, despite potential
high-pruning rate drawbacks.
Viviana Crescitelli et al.[3] propose a filter-level pruning
method for DNNs. This approach optimizes inference speed
and resource efficiency on edge devices with minimal accuracy
loss but requires careful tuning of layer-specific pruning ratios.
Yongqi An et al.[1] introduce a retraining-free structured
pruning framework for Large Language Models (LLMs). Their
method incorporates a fluctuation-based pruning metric, adap-
tive structure search, and a bias compensation mechanism to
achieve efficient pruning. FLAP significantly enhances infer-
ence speed and reduces model size without requiring retrain-
ing, outperforming state-of-the-art methods like LLM-Pruner
and Wanda-sp on LLaMA models across various benchmarks.
From referring to these research papers, we derived an
idea for pruning on edge devices. Additionally, our proposed
solution is inspired by the research of Jaron Maene et al.[10],
which suggests that sparse subnetworks in dense networks can
achieve similar accuracy when retrained. This study demon-
strates stable training with linear mode connectivity, support-
ing the idea that lottery tickets retrain to similar regions, but
questions their independence from dense training and iterative
pruning.
Eran Malach et al.[11] further strengthen the lottery ticket
hypothesis by proving that over-parameterized neural networks
with random weights always contain a subnetwork matching
the accuracy of a target network, without additional training.
IV. NEUROLOGICAL PRUNING APPROACH
The Neurological Pruning Algorithm is designed to opti-
mize neural networks by iteratively pruning less important
weights while maintaining high performance. Unlike tradi-
tional methods, this approach incorporates adaptive, layer-
specific pruning thresholds that dynamically adjust based
on weight activations, ensuring efficient model optimization
across various stages of training. Drawing inspiration from
the Lottery Ticket Hypothesis, which suggests that sparse
subnetworks can perform similarly to their dense counterparts
when retrained, the algorithm selectively deactivates weights
using binary masks to enhance computational efficiency with-
out sacrificing accuracy.
The algorithm operates in several epochs, during which
weight activations are evaluated. Based on a quantile-based
strategy, pruning thresholds are established for each layer, en-
suring that only weights contributing minimally to the model’s
output are removed. After pruning, the model undergoes fine-
tuning to restore any lost accuracy and further optimize the
network. This step enhances the adaptability and scalability
of the pruning method, making it suitable for deployment in
resource-constrained environments
A. Key Characteristics and Advantages
The Neurological Pruning Algorithm boasts several key
characteristics that distinguish it from conventional pruning
techniques. One notable feature is its layer-specific pruning,
which evaluates the importance of weights individually for
each layer, leading to a more nuanced and efficient sparsifi-
cation process. This approach prevents the model from losing
essential features that may otherwise be pruned by a global
approach.
The algorithm’s use of quantile-based thresholding en-
sures that only weights with the least contribution to the
model’s output are pruned. This fine-tuned method avoids
performance degradation by carefully targeting weights that
are less significant to the network’s overall functionality.
Furthermore, the pruning process is iterative, allowing for
continuous model evaluation and real-time adjustments based
on performance feedback, ensuring that accuracy is retained
even as sparsity increases.
Additionally, the algorithm maintains a careful balance
between sparsity and accuracy, enabling a reduction in the
model’s computational complexity while preserving its predic-
tive capabilities. This characteristic is particularly important
for applications requiring rapid inference times, such as in
real-time healthcare systems and edge computing scenarios,
where both energy efficiency and computational power are
crucial.
B. Impact on Model Performance
The Neurological Pruning Algorithm significantly improves
the efficiency of neural networks, making them more suitable
for deployment on edge devices. Key performance metrics
such as compression ratio (CR) and accuracy retention
(AR) are critical indicators of the algorithm’s effectiveness.
For instance, in evaluating a ResNet-18 model, the algorithm
achieved a CR of 1, indicating that while the number of
parameters remained the same, the model’s efficiency im-
proved through selective pruning, reducing computational load
without sacrificing performance.
Moreover, the algorithm demonstrates the ability to achieve
high accuracy retention, with models maintaining or even
slightly improving their predictive performance post-pruning.
This outcome highlights the algorithm’s capability to effi-
ciently remove redundant weights while preserving the core
functionality of the network. The result is a model that is
not only more computationally efficient but also adaptable
across different architectures and datasets, making it suit-
able for a wide range of applications, from image classification
to natural language processing .
In practical terms, the Neurological Pruning Algorithm en-
ables faster inference times and lower energy consumption,
two crucial aspects for deploying AI models in resource-
constrained environments. As such, it holds significant poten-
tial for improving the viability of real-time, energy-efficient
AI applications, particularly in fields like healthcare, where
both speed and sustainability are essential.
V. NEUROLOGICAL PRUNING ALGORITHM
The Neurological Pruning Algorithm is a structured and
adaptive methodology designed to optimize neural networks
for resource-constrained environments. This algorithm goes
beyond traditional pruning approaches by incorporating layer-
specific pruning thresholds that dynamically adjust to the
importance of weight activations within each layer. Drawing
on principles inspired by the Lottery Ticket Hypothesis and re-
inforcement learning, the algorithm strikes an optimal balance
between model sparsity and accuracy .
A. Algorithm Description
The Neurological Pruning Algorithm iteratively refines the
neural network through a series of training epochs. During
each epoch, weight activations are tracked, normalized, and
analyzed to identify low-importance weights. These weights
are selectively deactivated using binary masks, ensuring com-
putational efficiency without compromising the network’s pre-
dictive performance. The algorithm further integrates a fine-
tuning phase, enabling the pruned network to recover and
enhance its accuracy. The detailed process is presented in
Algorithm 1.
By dynamically adjusting to layer-specific characteristics
and maintaining performance metrics, the Neurological Prun-
ing Algorithm ensures a significant reduction in computational
and memory requirements, paving the way for efficient deploy-
ment of deep learning models on edge devices .
B. Input Parameters and Output
Inputs:
• Pretrained model (M)
• Dataset (D)
• Prune fraction (p)
• Total epochs (E)
Output:
• Pruned and fine-tuned model (Mpruned)
C. Key Features and Advantages
The Neurological Pruning Algorithm is designed with sev-
eral distinctive features that contribute to its effectiveness
in optimizing neural network performance. These features
include:
Algorithm 1 Simplified Neurological Pruning Algorithm
Require: Pretrained model M, Dataset D, Prune fraction p,
Total epochs E
Ensure: Pruned and fine-tuned model Mpruned
1: Initialize binary masks M with all ones
2: for each epoch e = 1 to E do
3:
Compute weight activations A during forward passes
4:
for each layer i in M do
5:
Compute pruning threshold Ti as p-quantile of Ai
6:
Update mask Mi: retain weights with Ai > Ti
7:
Apply mask to layer: θi ←θi · Mi
8:
end for
9:
Evaluate model performance and update metrics
10: end for
11: Fine-tune Mpruned on D
12: return Mpruned
• Layer-specific Pruning: The algorithm applies pruning
on a per-layer basis, evaluating the weight activations
within each layer individually. This approach ensures that
the sparsity introduced by pruning is well-distributed and
context-dependent, facilitating more nuanced control over
model optimization .
• Quantile-based Thresholding: A quantile-based strat-
egy is employed to determine the pruning threshold for
each layer. This method ensures that only weights with
minimal contribution to the network’s output are pruned,
thereby reducing the risk of adversely affecting model
accuracy while enhancing sparsity .
• Iterative Pruning and Evaluation: The pruning pro-
cess follows an iterative approach, enabling continuous
assessment of the model’s performance after each pruning
step. This ensures dynamic adjustments to maintain a bal-
ance between model sparsity and predictive performance
throughout training.
• Preservation of Model Accuracy: By adjusting pruning
thresholds in response to ongoing performance metrics,
the algorithm ensures that model accuracy is not com-
promised. The careful tuning of these thresholds allows
for significant sparsity while maintaining the robustness
of the network’s predictive capabilities .
D. Impact on Neural Network Performance
The Neurological Pruning Algorithm has been extensively
evaluated across various benchmark datasets and model archi-
tectures, demonstrating its capacity to achieve the following
outcomes:
• High Sparsity with Minimal Accuracy Loss: The algo-
rithm effectively prunes unnecessary weights, achieving
high sparsity levels without causing significant degrada-
tion in model accuracy. This outcome is critical for the
efficient deployment of models in resource-constrained
environments.
• Reduced Computational Complexity: By sparsifying
the weight matrices, the algorithm reduces the compu-
tational load during inference, which translates to faster
model execution and lower resource consumption. This
characteristic is especially beneficial in edge computing
and real-time applications .
• Broad Applicability: The algorithm is highly flexible
and can be adapted to various neural network architec-
tures and datasets, making it suitable for a wide range
of tasks, from image recognition to natural language
processing .
VI. EVALUATION METRICS
A. Performance Comparison
Fig. 1. Evaluation of Neurological Pruning on ResNet-18 Model.
TABLE I
COMPARISON OF METRICS FOR UNPRUNED AND PRUNED MODELS
Metric
Unpruned
Model
Pruned
Model
Accuracy
0.9562
0.9583
Train Loss
0.1414
0.1625
Inference
Time
(s)
0.0318
0.0314
No of Parame-
ters(million)
11.7
11.7
B. Evaluation Metrics Calculation
The key metrics used to evaluate the proposed pruning
technique are as follows:
Compression Ratio (CR): The Compression Ratio (CR) is
calculated as:
CR = Total Parameters After Pruning
Total Parameters Before Pruning
(1)
CR = 11.7M
11.7M = 1
Accuracy Retention (AR): The Accuracy Retention (AR) is
calculated as:
AR =
Accuracy After Pruning −Baseline Accuracy
Baseline Accuracy

×100
(2)
AR =
0.958284 −0.956206
0.956206

× 100 = 0.22%
This result indicates that the pruned model achieves an ac-
curacy improvement of 0.22% when compared to unpruned
model.
C. Illustration of Results
Fig. 2. Graphical Representation of Neurological Pruning Results on ResNet-
18.
D. Discussion
The evaluation metrics highlight the effectiveness of the
proposed Neurological Pruning method on the ResNet-18
model. The Compression Ratio (CR) remains 1, indicating
no reduction in the total number of parameters, which suggests
that this approach optimizes the neural network by selectively
pruning connections rather than reducing the overall parameter
count. The Accuracy Retention (AR), which is approximately
0.22%, demonstrates that the Neurological Pruning technique
successfully improved the accuracy compared to the baseline
model.
These results validate the potential of Neurological Pruning
as an innovative approach to optimizing deep learning models.
By maintaining accuracy while likely enhancing sparsity and
computational efficiency, this method presents a promising di-
rection for achieving high-performance models under resource
constraints.
VII. CONCLUSION
In summary, the Neurological Pruning Algorithm provides
a systematic and principled approach to optimizing neural
networks. By balancing the trade-off between sparsity and
accuracy, it enables significant reductions in computational
complexity without sacrificing the network’s predictive per-
formance. Future work will focus on extending the algorithm
to dynamic network architectures and incorporating advanced
regularization techniques to further enhance its robustness and
adaptability in diverse applications.
REFERENCES
[1] Yongqi An, Xu Zhao, Tao Yu, Ming Tang, and Jinqiao Wang.
Fluctuation-based Adaptive Structured Pruning for Large Language
Models.
[2] Kyoungtaek Choi, Seong Min Wi, Ho Gi Jung, and Jae Kyu Suhr.
Simplification of Deep Neural Network-Based Object Detector for Real-
Time Edge Computing.
[3] Viviana Crescitelli, Seiji Miura, Goichi Ono, and Naohiro Kohmu. Edge
Devices Object Detection by Filter Pruning.
[4] Giacomo Di Fabrizio, Lorenzo Calisti, Chiara Contoli, Nicholas Kania,
and Emanuele Lattanzi. A Study on the Energy-Efficiency of the Object
Tracking Algorithms in Edge Devices.
[5] Yong He, Lin Lu, Hanjie Yuan, Guangxian Ye, Tianhang Jiang, Liang
Chen, Haiao Tan, Gaofeng Liao, Yanchao Zeng, and Jiawei Zhang. A
Compression Method for Object Detection Network for Edge Devices.
[6] Jielei Wang, Zongyong Cui, Zhipeng Zang, Xiangjie Meng, and Zongjie
Cao. Absorption Pruning of Deep Neural Network for Object Detection
in Remote Sensing Imagery.
[7] Satoru Koda, Alon Zolfi, Edita Grolman, Asaf Shabtai, Ikuya Morikawa,
and Yuval Elovici. Pros and Cons of Weight Pruning for Out-of-
Distribution Detection: An Empirical Survey.
[8] Shvetha S Kumar, Reshma R Nayak, Jismi S Kannampuzha, Jeeho Ryoo,
Sahil Rai, and Lizy K John. Evaluation of Pruning Techniques.
[9] Liang Li and Pengfei Zhao. Protective Self-Adaptive Pruning to Better
Compress DNNs.
[10] Jaron Maene, Mingxiao Li, and Marie-Francine Moens. Towards Un-
derstanding Iterative Magnitude Pruning: Why Lottery Tickets Win.
[11] Eran Malach, Gilad Yehudai, Shai Shalev-Shwartz, and Ohad Shamir.
Proving the Lottery Ticket Hypothesis: Pruning is All You Need.
[12] Jan M¨uller and Adrian Pigors. Efficient Multi-Object Tracking on Edge
Devices via Reconstruction-Based Channel Pruning.
[13] A. Arun Prakash and Karthik K. Srinivasan. Pruning Algorithms to
Determine Reliable Paths on Networks with Random and Correlated
Link Travel Times.
[14] Jose Reena K, Charul Nigam, G. Kirubasri, S. Jayachitra, Anurag Aeron,
and D. Suganthi. Real-Time Object Detection on Edge Devices Using
Mobile Neural Networks.
[15] Xubing Ye, Yukang Gan, Yixiao Ge, Xiao-Ping Zhang, and Yansong
Tang. ATP-LLaVA: Adaptive Token Pruning for Large Vision Language
Models.
[16] Shihao Zhang, Hekai Yang, Chunhua Yang, Wenxia Yuan, Xinghui
Li, Xinghua Wang, Yinsong Zhang, Xiaobo Cai, Yubo Sheng, Xiujuan
Deng, Wei Huang, Lei Li, Junjie He, and Baijuan Wang. Edge Device
Detection of Tea Leaves with One Bud and Two Leaves Based on
ShuffleNetv2-YOLOv5-Lite-E.
Neuro Prune: An Adaptive Approach for Efficient
Deep Neural Network Optimization on Edge
Devices
Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
1Professor, Department of Computer Science and Engineering, PSG College of Technology
grk.cse@psgtech.ac.in
2,3,4,5BE-CSE (AI&ML), PSG College of Technology
adhishthesak@gmail.com, mohanakumarp2828@gmail.com
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
Abstract—The optimization of deep neural networks for de-
ployment on edge devices is a significant research area due to
the demand for applications such as augmented reality, smart
cameras, and autonomous navigation. However, deploying large
deep learning models on edge devices poses challenges related to
computational power, energy consumption, and latency. Pruning
is a method to reduce the model size, accelerate inference, and
save power. The objective of the paper is to propose the Neuro
Prune algorithm and to apply it for the optimization of deep
neural networks on edge devices. Efforts have been made to
compare pruned and unpruned models. As a result, the pruned
model has an accuracy increase of 0.22%.
I. INTRODUCTION
The rise of intelligent systems, ranging from autonomous
drones to augmented reality (AR) devices and smart surveil-
lance cameras, has intensified the need for efficient deep neural
network deployment on edge devices with constrained com-
putational resources. While advanced deep learning models
like YOLO, SSD, ResNet, and MobileNet excel in accuracy,
their deployment on such devices is hindered by significant
computational demands, large memory footprints, and high
energy consumption. These challenges pose critical barriers to
real-time applications and sustainable deployment.
This paper builds on existing pruning techniques, includ-
ing unstructured pruning, structured pruning, and approaches
inspired by quantization, highlighting their strengths and lim-
itations. In response to the challenges identified, we introduce
Neuro Prune, a novel pruning approach that integrates princi-
ples from reinforcement learning and the Lottery Ticket Hy-
pothesis to achieve an optimal balance between accuracy, com-
putational efficiency, and energy consumption. By integrating
these principles, Neuro Prune provides a robust framework
for optimizing deep neural networks for deployment on edge
devices without sacrificing performance.
Neuro Prune reduces model size and computational re-
quirements while maintaining or even improving the model’s
accuracy. These findings underscore the potential of Neuro
Prune to enable real-time, energy-efficient deep learning on
resource-constrained devices.
Neuro Prune employs a systematic process to optimize
neural networks for edge deployment:
• Mask Initialization: Each layer’s weights are paired with
a binary mask (original_mask), initialized to ones.
• Activation Tracking: During forward passes over the
dataset, the magnitudes of weight activations are mon-
itored and recorded.
• Normalize and Prune: The recorded activations are
normalized, and a threshold based on the pruning fraction
is computed. Using this threshold, the masks are dynam-
ically updated to prune unimportant weights.
• Evaluate and Log Metrics: After pruning, metrics such
as model sparsity and accuracy are computed. A reward
metric is derived to balance accuracy and sparsity using
a tunable parameter, λweight.
• Retrain the Model: The pruned model undergoes retrain-
ing to improve the loss in accuracy while maintaining low
computational requirements
Neuro Prune’s unique combination of reinforcement learn-
ing for adaptive decision-making and the Lottery Ticket Hy-
pothesis for identifying optimal subnetworks enables it to
achieve remarkable efficiency. This approach ensures substan-
tial reduction in model size and energy consumption while
preserving accuracy and inference speed, making it an ideal
solution for edge-based deep learning applications.
Through extensive empirical evaluations and theoretical
insights, this paper demonstrates how Neuro Pruning outper-
forms traditional pruning methods, setting a new benchmark
for sustainable, high-performance AI deployment on edge
devices. By situating our work within the broader research
context, we aim to offer a critical contribution to the ongoing
evolution of neural network optimization techniques.
II. BACKGROUND
A. Need for Pruning Techniques
Pruning is a crucial optimization technique for deploying
deep neural networks in resource-constrained environments
like healthcare. For example, wearable devices like smart-
watches that monitor heart rates or portable EEG systems
for brain activity analysis require lightweight models due to
their limited computational power and memory. Pruning helps
address these constraints by reducing the memory footprint
and computational load, enabling efficient model deployment.
Additionally, it minimizes energy consumption, which is vital
for sustainable and real-time healthcare applications. Faster
inference enabled by pruning is particularly beneficial for
time-sensitive tasks like seizure detection or emergency di-
agnostics. Moreover, pruning enhances model interpretability
by retaining only the most critical components, which is
essential for gaining trust and ensuring regulatory compliance
in healthcare. By reducing the hardware and operational costs,
pruning also makes deploying advanced AI solutions on edge
devices more cost-effective and accessible.
B. Types of Pruning Techniques
Pruning methods are broadly classified based on granu-
larity, timing, and approach. Granularity-based pruning in-
cludes structured pruning, which removes entire components
such as neurons or filters to streamline computations, and
unstructured pruning, which eliminates individual weights
for finer optimization, though it often requires specialized
hardware. Timing-based pruning involves pre-training pruning,
which optimizes models before training; post-training prun-
ing, where redundant components are removed from trained
models with possible retraining; and dynamic pruning during
training, which adjusts models in real-time for improved
adaptability. Metric-based pruning uses parameters like weight
magnitude or gradient contribution to identify and remove
less critical components, while application-specific pruning
tailors strategies to specific tasks, such as optimizing for EEG
signal analysis, or hardware constraints like GPUs. Together,
these methods enable the development of efficient, scalable,
and task-specific models, particularly suited for edge device
deployment in healthcare and other domains.
III. RELATED WORK
Jielei Wang et al.[12] introduced an absorption pruning
method for object detection in remote sensing imagery. This
study achieves efficient compression with minimal accuracy
loss but requires careful layer-wise pruning ratio tuning.
Jan Muller et al., in their research[12], propose a neural
network pruning method for multi-object tracking (MOT) that
reduces model size by up to 70
Liang Li et al., in their work[9], introduce a novel pruning
method for DNNs that utilizes a self-adaptive mechanism
based on weight sparsity ratios and a protective reconstruction
mechanism. Their approach improves both model compres-
sion and accuracy, outperforming state-of-the-art methods on
CIFAR-10 and ImageNet datasets.
Giacomo Di Fabrizio et al.[4] discuss integrating machine
learning with edge devices like Google Coral AI and Nvidia
Jetson Nano. Their work revolutionizes computer vision and
real-time tracking but raises energy concerns due to the
computational demands of advanced algorithms.
Satoru Koda et al.[7] explore how weight pruning in
DNNs enhances out-of-distribution (OOD) detection, particu-
larly with Mahalanobis-based approaches, by improving global
feature extraction and leveraging weights not critical for
classification.
Shvetha S Kumar et al. analyze and compare three pruning
techniques—L1-norm filter pruning, channel pruning, and
weight pruning—on CNNs for accuracy and inference time,
highlighting greater speedup on NVIDIA V100 GPUs com-
pared to GTX1080 Ti in their study[8].
A Arun Prakash et al. propose a sub-path pruning algorithm
for solving the Minimum Robust-Cost Path (MRCP) problem
on stochastic networks, extending it to related problems and
demonstrating its efficiency on real-world networks in their
study[13].
Kyoungtaek Choi et al.[2] show that Quantization-Aware
Training outperforms Post-Training Quantization, and sparsity
training with Optimal Reduction pruning improves YOLOv4’s
efficiency and edge-device performance, despite potential
high-pruning rate drawbacks.
Viviana Crescitelli et al.[3] propose a filter-level pruning
method for DNNs. This approach optimizes inference speed
and resource efficiency on edge devices with minimal accuracy
loss but requires careful tuning of layer-specific pruning ratios.
Yongqi An et al.[1] introduce a retraining-free structured
pruning framework for Large Language Models (LLMs). Their
method incorporates a fluctuation-based pruning metric, adap-
tive structure search, and a bias compensation mechanism to
achieve efficient pruning. FLAP significantly enhances infer-
ence speed and reduces model size without requiring retrain-
ing, outperforming state-of-the-art methods like LLM-Pruner
and Wanda-sp on LLaMA models across various benchmarks.
From referring to these research papers, we derived an
idea for pruning on edge devices. Additionally, our proposed
solution is inspired by the research of Jaron Maene et al.[10],
which suggests that sparse subnetworks in dense networks can
achieve similar accuracy when retrained. This study demon-
strates stable training with linear mode connectivity, support-
ing the idea that lottery tickets retrain to similar regions, but
questions their independence from dense training and iterative
pruning.
Eran Malach et al.[11] further strengthen the lottery ticket
hypothesis by proving that over-parameterized neural networks
with random weights always contain a subnetwork matching
the accuracy of a target network, without additional training.
IV. NEUROLOGICAL PRUNING APPROACH
The Neurological Pruning Algorithm is designed to opti-
mize neural networks by iteratively pruning less important
weights while maintaining high performance. Unlike tradi-
tional methods, this approach incorporates adaptive, layer-
specific pruning thresholds that dynamically adjust based
on weight activations, ensuring efficient model optimization
across various stages of training. Drawing inspiration from
the Lottery Ticket Hypothesis, which suggests that sparse
subnetworks can perform similarly to their dense counterparts
when retrained, the algorithm selectively deactivates weights
using binary masks to enhance computational efficiency with-
out sacrificing accuracy.
The algorithm operates in several epochs, during which
weight activations are evaluated. Based on a quantile-based
strategy, pruning thresholds are established for each layer, en-
suring that only weights contributing minimally to the model’s
output are removed. After pruning, the model undergoes fine-
tuning to restore any lost accuracy and further optimize the
network. This step enhances the adaptability and scalability
of the pruning method, making it suitable for deployment in
resource-constrained environments
A. Key Characteristics and Advantages
The Neurological Pruning Algorithm boasts several key
characteristics that distinguish it from conventional pruning
techniques. One notable feature is its layer-specific pruning,
which evaluates the importance of weights individually for
each layer, leading to a more nuanced and efficient sparsifi-
cation process. This approach prevents the model from losing
essential features that may otherwise be pruned by a global
approach.
The algorithm’s use of quantile-based thresholding en-
sures that only weights with the least contribution to the
model’s output are pruned. This fine-tuned method avoids
performance degradation by carefully targeting weights that
are less significant to the network’s overall functionality.
Furthermore, the pruning process is iterative, allowing for
continuous model evaluation and real-time adjustments based
on performance feedback, ensuring that accuracy is retained
even as sparsity increases.
Additionally, the algorithm maintains a careful balance
between sparsity and accuracy, enabling a reduction in the
model’s computational complexity while preserving its predic-
tive capabilities. This characteristic is particularly important
for applications requiring rapid inference times, such as in
real-time healthcare systems and edge computing scenarios,
where both energy efficiency and computational power are
crucial.
B. Impact on Model Performance
The Neurological Pruning Algorithm significantly improves
the efficiency of neural networks, making them more suitable
for deployment on edge devices. Key performance metrics
such as compression ratio (CR) and accuracy retention
(AR) are critical indicators of the algorithm’s effectiveness.
For instance, in evaluating a ResNet-18 model, the algorithm
achieved a CR of 1, indicating that while the number of
parameters remained the same, the model’s efficiency im-
proved through selective pruning, reducing computational load
without sacrificing performance.
Moreover, the algorithm demonstrates the ability to achieve
high accuracy retention, with models maintaining or even
slightly improving their predictive performance post-pruning.
This outcome highlights the algorithm’s capability to effi-
ciently remove redundant weights while preserving the core
functionality of the network. The result is a model that is
not only more computationally efficient but also adaptable
across different architectures and datasets, making it suit-
able for a wide range of applications, from image classification
to natural language processing .
In practical terms, the Neurological Pruning Algorithm en-
ables faster inference times and lower energy consumption,
two crucial aspects for deploying AI models in resource-
constrained environments. As such, it holds significant poten-
tial for improving the viability of real-time, energy-efficient
AI applications, particularly in fields like healthcare, where
both speed and sustainability are essential.
V. NEUROLOGICAL PRUNING ALGORITHM
The Neurological Pruning Algorithm is a structured and
adaptive methodology designed to optimize neural networks
for resource-constrained environments. This algorithm goes
beyond traditional pruning approaches by incorporating layer-
specific pruning thresholds that dynamically adjust to the
importance of weight activations within each layer. Drawing
on principles inspired by the Lottery Ticket Hypothesis and re-
inforcement learning, the algorithm strikes an optimal balance
between model sparsity and accuracy .
A. Algorithm Description
The Neurological Pruning Algorithm iteratively refines the
neural network through a series of training epochs. During
each epoch, weight activations are tracked, normalized, and
analyzed to identify low-importance weights. These weights
are selectively deactivated using binary masks, ensuring com-
putational efficiency without compromising the network’s pre-
dictive performance. The algorithm further integrates a fine-
tuning phase, enabling the pruned network to recover and
enhance its accuracy. The detailed process is presented in
Algorithm 1.
By dynamically adjusting to layer-specific characteristics
and maintaining performance metrics, the Neurological Prun-
ing Algorithm ensures a significant reduction in computational
and memory requirements, paving the way for efficient deploy-
ment of deep learning models on edge devices .
B. Input Parameters and Output
Inputs:
• Pretrained model (M)
• Dataset (D)
• Prune fraction (p)
• Total epochs (E)
Output:
• Pruned and fine-tuned model (Mpruned)
C. Key Features and Advantages
The Neurological Pruning Algorithm is designed with sev-
eral distinctive features that contribute to its effectiveness
in optimizing neural network performance. These features
include:
Algorithm 1 Simplified Neurological Pruning Algorithm
Require: Pretrained model M, Dataset D, Prune fraction p,
Total epochs E
Ensure: Pruned and fine-tuned model Mpruned
1: Initialize binary masks M with all ones
2: for each epoch e = 1 to E do
3:
Compute weight activations A during forward passes
4:
for each layer i in M do
5:
Compute pruning threshold Ti as p-quantile of Ai
6:
Update mask Mi: retain weights with Ai > Ti
7:
Apply mask to layer: θi ←θi · Mi
8:
end for
9:
Evaluate model performance and update metrics
10: end for
11: Fine-tune Mpruned on D
12: return Mpruned
• Layer-specific Pruning: The algorithm applies pruning
on a per-layer basis, evaluating the weight activations
within each layer individually. This approach ensures that
the sparsity introduced by pruning is well-distributed and
context-dependent, facilitating more nuanced control over
model optimization .
• Quantile-based Thresholding: A quantile-based strat-
egy is employed to determine the pruning threshold for
each layer. This method ensures that only weights with
minimal contribution to the network’s output are pruned,
thereby reducing the risk of adversely affecting model
accuracy while enhancing sparsity .
• Iterative Pruning and Evaluation: The pruning pro-
cess follows an iterative approach, enabling continuous
assessment of the model’s performance after each pruning
step. This ensures dynamic adjustments to maintain a bal-
ance between model sparsity and predictive performance
throughout training.
• Preservation of Model Accuracy: By adjusting pruning
thresholds in response to ongoing performance metrics,
the algorithm ensures that model accuracy is not com-
promised. The careful tuning of these thresholds allows
for significant sparsity while maintaining the robustness
of the network’s predictive capabilities .
D. Impact on Neural Network Performance
The Neurological Pruning Algorithm has been extensively
evaluated across various benchmark datasets and model archi-
tectures, demonstrating its capacity to achieve the following
outcomes:
• High Sparsity with Minimal Accuracy Loss: The algo-
rithm effectively prunes unnecessary weights, achieving
high sparsity levels without causing significant degrada-
tion in model accuracy. This outcome is critical for the
efficient deployment of models in resource-constrained
environments.
• Reduced Computational Complexity: By sparsifying
the weight matrices, the algorithm reduces the compu-
tational load during inference, which translates to faster
model execution and lower resource consumption. This
characteristic is especially beneficial in edge computing
and real-time applications .
• Broad Applicability: The algorithm is highly flexible
and can be adapted to various neural network architec-
tures and datasets, making it suitable for a wide range
of tasks, from image recognition to natural language
processing .
VI. EVALUATION METRICS
A. Performance Comparison
Fig. 1. Evaluation of Neurological Pruning on ResNet-18 Model.
TABLE I
COMPARISON OF METRICS FOR UNPRUNED AND PRUNED MODELS
Metric
Unpruned
Model
Pruned
Model
Accuracy
0.9562
0.9583
Train Loss
0.1414
0.1625
Inference
Time
(s)
0.0318
0.0314
No of Parame-
ters(million)
11.7
11.7
B. Evaluation Metrics Calculation
The key metrics used to evaluate the proposed pruning
technique are as follows:
Compression Ratio (CR): The Compression Ratio (CR) is
calculated as:
CR = Total Parameters After Pruning
Total Parameters Before Pruning
(1)
CR = 11.7M
11.7M = 1
Accuracy Retention (AR): The Accuracy Retention (AR) is
calculated as:
AR =
Accuracy After Pruning −Baseline Accuracy
Baseline Accuracy

×100
(2)
AR =
0.958284 −0.956206
0.956206

× 100 = 0.22%
This result indicates that the pruned model achieves an ac-
curacy improvement of 0.22% when compared to unpruned
model.
C. Illustration of Results
Fig. 2. Graphical Representation of Neurological Pruning Results on ResNet-
18.
D. Discussion
The evaluation metrics highlight the effectiveness of the
proposed Neurological Pruning method on the ResNet-18
model. The Compression Ratio (CR) remains 1, indicating
no reduction in the total number of parameters, which suggests
that this approach optimizes the neural network by selectively
pruning connections rather than reducing the overall parameter
count. The Accuracy Retention (AR), which is approximately
0.22%, demonstrates that the Neurological Pruning technique
successfully improved the accuracy compared to the baseline
model.
These results validate the potential of Neurological Pruning
as an innovative approach to optimizing deep learning models.
By maintaining accuracy while likely enhancing sparsity and
computational efficiency, this method presents a promising di-
rection for achieving high-performance models under resource
constraints.
VII. CONCLUSION
In summary, the Neurological Pruning Algorithm provides
a systematic and principled approach to optimizing neural
networks. By balancing the trade-off between sparsity and
accuracy, it enables significant reductions in computational
complexity without sacrificing the network’s predictive per-
formance. Future work will focus on extending the algorithm
to dynamic network architectures and incorporating advanced
regularization techniques to further enhance its robustness and
adaptability in diverse applications.
REFERENCES
[1] Yongqi An, Xu Zhao, Tao Yu, Ming Tang, and Jinqiao Wang.
Fluctuation-based Adaptive Structured Pruning for Large Language
Models.
[2] Kyoungtaek Choi, Seong Min Wi, Ho Gi Jung, and Jae Kyu Suhr.
Simplification of Deep Neural Network-Based Object Detector for Real-
Time Edge Computing.
[3] Viviana Crescitelli, Seiji Miura, Goichi Ono, and Naohiro Kohmu. Edge
Devices Object Detection by Filter Pruning.
[4] Giacomo Di Fabrizio, Lorenzo Calisti, Chiara Contoli, Nicholas Kania,
and Emanuele Lattanzi. A Study on the Energy-Efficiency of the Object
Tracking Algorithms in Edge Devices.
[5] Yong He, Lin Lu, Hanjie Yuan, Guangxian Ye, Tianhang Jiang, Liang
Chen, Haiao Tan, Gaofeng Liao, Yanchao Zeng, and Jiawei Zhang. A
Compression Method for Object Detection Network for Edge Devices.
[6] Jielei Wang, Zongyong Cui, Zhipeng Zang, Xiangjie Meng, and Zongjie
Cao. Absorption Pruning of Deep Neural Network for Object Detection
in Remote Sensing Imagery.
[7] Satoru Koda, Alon Zolfi, Edita Grolman, Asaf Shabtai, Ikuya Morikawa,
and Yuval Elovici. Pros and Cons of Weight Pruning for Out-of-
Distribution Detection: An Empirical Survey.
[8] Shvetha S Kumar, Reshma R Nayak, Jismi S Kannampuzha, Jeeho Ryoo,
Sahil Rai, and Lizy K John. Evaluation of Pruning Techniques.
[9] Liang Li and Pengfei Zhao. Protective Self-Adaptive Pruning to Better
Compress DNNs.
[10] Jaron Maene, Mingxiao Li, and Marie-Francine Moens. Towards Un-
derstanding Iterative Magnitude Pruning: Why Lottery Tickets Win.
[11] Eran Malach, Gilad Yehudai, Shai Shalev-Shwartz, and Ohad Shamir.
Proving the Lottery Ticket Hypothesis: Pruning is All You Need.
[12] Jan M¨uller and Adrian Pigors. Efficient Multi-Object Tracking on Edge
Devices via Reconstruction-Based Channel Pruning.
[13] A. Arun Prakash and Karthik K. Srinivasan. Pruning Algorithms to
Determine Reliable Paths on Networks with Random and Correlated
Link Travel Times.
[14] Jose Reena K, Charul Nigam, G. Kirubasri, S. Jayachitra, Anurag Aeron,
and D. Suganthi. Real-Time Object Detection on Edge Devices Using
Mobile Neural Networks.
[15] Xubing Ye, Yukang Gan, Yixiao Ge, Xiao-Ping Zhang, and Yansong
Tang. ATP-LLaVA: Adaptive Token Pruning for Large Vision Language
Models.
[16] Shihao Zhang, Hekai Yang, Chunhua Yang, Wenxia Yuan, Xinghui
Li, Xinghua Wang, Yinsong Zhang, Xiaobo Cai, Yubo Sheng, Xiujuan
Deng, Wei Huang, Lei Li, Junjie He, and Baijuan Wang. Edge Device
Detection of Tea Leaves with One Bud and Two Leaves Based on
ShuffleNetv2-YOLOv5-Lite-E.
""", "metadata": {"source": "geo_facts.txt", "page": 1}},
    {"content": "The Great Wall of China is located in China and is a famous landmark.", "metadata": {"source": "world_landmarks.txt", "page": 5}},
    {"content": "Paris is known for the Eiffel Tower.", "metadata": {"source": "geo_facts.txt", "page": 2}},
]

# Convert raw data to LangChain Document objects
documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in documents_data]

# Create FAISS vector store directly from documents
# This handles embedding the texts and building the index internally
try:
    vector_store = FAISS.from_documents(documents, embedding_model)
    print("FAISS vector store created successfully.")
except Exception as e:
    print(f"Error creating FAISS vector store: {e}")
# Exit if vector store creation fails

# Step 4: Load Gemini model for text generation

# Step 5: Create the Retrieval-Augmented Generation pipeline
print("Creating RAG chain...")
retriever = vector_store.as_retriever(
    search_type="similarity", # Other options: "mmr", "similarity_score_threshold"
    search_kwargs={'k': 2} # Retrieve top 2 relevant documents
)

# Use the recommended from_chain_type method
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # Options: "stuff", "map_reduce", "refine", "map_rerank"
                        # "stuff" puts all retrieved docs into the context
    retriever=retriever,
    return_source_documents=True # Include source documents in the output
    # chain_type_kwargs={"prompt": YOUR_CUSTOM_PROMPT} # Optional: customize prompt
)
print("RAG chain created.")

query = "who is surya narayanaa"
print(f"\nQuerying the RAG system with: '{query}'")

try:
    result = rag_chain.invoke({"query": query})
    # Output results
    print("\n--- RAG System Output ---")
    print("Answer:", result.get("result", "No answer found."))
    print("\nSources:")
    if result.get("source_documents"):
        for i, doc in enumerate(result["source_documents"]):
            print(f"  Source {i+1}:")
            print(f"    Content: {doc.page_content}")
            print(f"    Metadata: {doc.metadata}")
    else:
        print("  No source documents found or returned.")
    print("--- End of Output ---")

except Exception as e:
    print(f"\nError during RAG chain execution: {e}")



Querying the RAG system with: 'who is surya narayanaa'

--- RAG System Output ---
Answer: Surya Narayanaa N T is listed as a co-author (5th author) of the research paper "Neuro Prune: An Adaptive Approach for Efficient Deep Neural Network Optimization on Edge Devices." He is a student (BE-CSE (AI&ML)) at PSG College of Technology. His email address is suryanarayanaant@gmail.com.

Sources:
  Source 1:
    Content: 
        Neuro Prune: An Adaptive Approach for Efficient
Deep Neural Network Optimization on Edge
Devices
Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
1Professor, Department of Computer Science and Engineering, PSG College of Technology
grk.cse@psgtech.ac.in
2,3,4,5BE-CSE (AI&ML), PSG College of Technology
adhishthesak@gmail.com, mohanakumarp2828@gmail.com
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
Abstract—The optimization of deep neural networks for de-
ployment on edge devices is a significant research area due to
the